# 1. Data Ingestion
This step will load the dataset and allow you to preview the first few rows to get an understanding of its structure.

In [1]:
import pandas as pd

# Load the dataset from the CSV file
data = pd.read_csv("D:/Microsoft Data Engineer/Graduation Project/online_retail.csv")

# Display the first few rows of the dataset
print(data.head())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  Total  CustomerID         Country  
0  12/1/2010 8:26       2.55  15.30     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39  20.34     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75  22.00     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39  20.34     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39  20.34     17850.0  United Kingdom  


# 2. Data Cleaning

2.1. Handling Missing Values (CustomerID):
Missing values are common in datasets like this. In e-commerce data, missing CustomerID is important because it's required for customer-level insights.


Why: Missing CustomerID can lead to inaccurate customer insights (e.g., segmentation, personalization).
Strategy: Remove rows with missing CustomerID, as those records won’t contribute meaningfully to customer-level analysis.

In [2]:
# Check for missing values
print(data.isnull().sum())

# Drop rows where CustomerID is missing
data_clean = data.dropna(subset=['CustomerID'])

# Confirm that no missing CustomerID values exist
print(data_clean.isnull().sum())


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
Total               0
CustomerID     135080
Country             0
dtype: int64
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Total          0
CustomerID     0
Country        0
dtype: int64


2.2. Removing Duplicates:
Duplicate records can skew data analysis, especially when counting transactions or total sales.

Why: Duplicate entries can inflate sales, number of transactions, and other key metrics.
Strategy: Remove all duplicate rows based on all columns to ensure only unique transactions are analyzed.

In [3]:
# Check for duplicate rows
duplicates = data_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Drop duplicate rows
data_clean = data_clean.drop_duplicates()

# Confirm duplicates are removed
print(data_clean.duplicated().sum())


Number of duplicate rows: 5225
0


2.3. Handling Incorrect or Negative Quantities:
Negative quantities typically indicate product returns, which may be useful, but in most cases, we filter them out for the purpose of sales analysis.

Why: Negative quantities often reflect returns, which may skew sales metrics (like revenue or number of items sold).
Strategy: Remove all transactions with negative quantities for standard sales analysis or keep them separate for return analysis.

In [4]:
# Check for negative quantities
negative_quantities = data_clean[data_clean['Quantity'] < 0]
print(f"Number of negative quantity transactions: {len(negative_quantities)}")

# Remove rows where quantity is negative
data_clean = data_clean[data_clean['Quantity'] > 0]

# Confirm no negative quantities exist
print(data_clean[data_clean['Quantity'] < 0])


Number of negative quantity transactions: 8872
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, Total, CustomerID, Country]
Index: []


2.4. Handling Invalid Stock Codes:
Sometimes stock codes may be placeholders or invalid (e.g., codes that don’t match valid products). You can filter for valid stock codes.

Why: Invalid stock codes often correspond to non-product-related transactions like charges, fees, or testing records.
Strategy: Identify and filter out invalid stock codes.

In [5]:
# Explore the unique stock codes to identify invalid ones
print(data_clean['StockCode'].unique())

# Example of filtering out invalid stock codes (replace 'INVALID' with any actual invalid codes identified)
invalid_stock_codes = ['POST', 'D', 'M', 'DOT', 'BANK CHARGES', 'PADS', 'CRUK']

data_clean = data_clean[~data_clean['StockCode'].isin(invalid_stock_codes)]

# Confirm invalid stock codes are removed
print(data_clean['StockCode'].isin(invalid_stock_codes).sum())


['85123A' '71053' '84406B' ... '90214Z' '90089' '23843']
0


# 3. Ensuring Consistency and Final Checks:
3.1. Fix Data Types:
Ensure that the columns have the correct data types (e.g., dates should be in datetime format, numbers should be numeric).

Why: Correct data types ensure that operations like date filtering, grouping, or aggregation work as expected.
Strategy: Ensure dates are datetime objects, IDs are strings, and numeric values are in the correct format.

In [6]:
# Convert InvoiceDate to datetime format
data_clean['InvoiceDate'] = pd.to_datetime(data_clean['InvoiceDate'])

# Ensure CustomerID is a string
data_clean['CustomerID'] = data_clean['CustomerID'].astype(str)

# Confirm data types
print(data_clean.dtypes)


InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
Total                 float64
CustomerID             object
Country                object
dtype: object


3.2. Recalculate Derived Columns:
You might need to create or adjust derived columns, such as TotalPrice.

Why: This provides a clearer view of transaction value, helping with sales forecasting and trend analysis.
Strategy: Derive columns like TotalPrice to capture relevant transaction metrics.

In [7]:
# Calculate total price for each transaction
data_clean['TotalPrice'] = data_clean['Quantity'] * data_clean['UnitPrice']

# Preview the result
print(data_clean[['Quantity', 'UnitPrice', 'TotalPrice']].head())


   Quantity  UnitPrice  TotalPrice
0         6       2.55       15.30
1         6       3.39       20.34
2         8       2.75       22.00
3         6       3.39       20.34
4         6       3.39       20.34


3.3. Final Checks and Exporting Cleaned Data:
Once cleaning is done, export the cleaned data for use in the next steps (e.g., loading into a data warehouse or machine learning pipeline).

Why: After cleaning, the data is ready for analysis, machine learning, or uploading to a cloud data warehouse (Azure SQL).
Strategy: Save the cleaned dataset for easy access.

In [8]:
# Final check on the cleaned data
print(data_clean.info())

# Export cleaned data to CSV
data_clean.to_csv('D:/Microsoft Data Engineer/Graduation Project/cleaned_online_retail.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 391316 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    391316 non-null  object        
 1   StockCode    391316 non-null  object        
 2   Description  391316 non-null  object        
 3   Quantity     391316 non-null  int64         
 4   InvoiceDate  391316 non-null  datetime64[ns]
 5   UnitPrice    391316 non-null  float64       
 6   Total        391316 non-null  float64       
 7   CustomerID   391316 non-null  object        
 8   Country      391316 non-null  object        
 9   TotalPrice   391316 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 32.8+ MB
None
